- capire come scrapare meglio user review counts
- join df, df2, (df3)
- pulizia dati (contare nulle, casting)
- script con funzioni???
- cartella scripts e data
- aggiungere table of contents
- commentare tutto

# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import os

# Data collection

IMDB doesn't have an official API. However, there are two unofficial APIs providing access to IMDB data: The Open Movie Database (OMDb) and The Movie Database (TMDB). In this project we'll rely on the latter as it doesn't enforce a rate limit. We're also going to supplement the API data with scraped data.

The TMDB API works by supplying the ID of the film(s) whose info we'd like to retrieve. The IDs consist of a double "t" followed by seven or eight digits. They can be found inside a film URL. For example, "tt0050782" is the ID for this film: https://www.imdb.com/title/tt0050782/. 

The first step will thus be retrieving the IDs of the 10 000 most popular films on IMDB, which can be found in this [playlist](https://www.imdb.com/search/keyword/?mode=detail&page=1&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc).

## Scraping the film IDs and the URLs

To scrape the film IDs of our 10 000 films we'll need to iterate a for loop over the first 200 pages in the playlist (each containing 50 films). The film ID is the value of the 'href' attribute inside h3 elements with the "lister-item-header" class. We'll write a simple regex to extract the film ID. We'll also scrape the entire URL: it's going to come in handy later when we do the scraping.

In [6]:
film_ids = []
film_urls = []

for i in range(1, 201):
  
  content = requests.get("https://www.imdb.com/search/keyword/?mode=detail&page=" + str(i) + "&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc").content
  
  soup = BeautifulSoup(content)

  for film in soup.find_all("h3", class_="lister-item-header"):

    link = film.find("a").get("href")
    film_id = re.findall(pattern = r"tt\d+", string = link)[0]
    film_ids.append(film_id)

    film_url = "https://www.imdb.com" + link
    film_urls.append(film_url)
    

## API calls

We'll now write a function to request the data from TMDB. The function takes a vector of film IDs as input and it returns a dataframe. For each ID it sends a GET request to the API. If the status code is equal to 200 (i.e. the request has been successful) it appends the data to a Pandas dataframe. Otherwise, it just appends the film_id and fills the remaining columns with null values.

In [7]:
def build_film_df(film_ids):
    
  api_key = os.environ.get("tmdb_api_key")

  df = pd.DataFrame(columns = ["id", "title", "release_date", "runtime", "country", "language", 
                               "genre", "studios", "budget", "revenue"])

  for film_id in film_ids:
      
      response = requests.get(f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}")

      if response.status_code == 200:  

        response_json = response.json()

        df = df.append({"id": response_json["imdb_id"],
                      "title": response_json["title"],
                      "release_date": response_json["release_date"],
                      "runtime": response_json["runtime"],
                      "country": ';'.join([country['name'] for country in response_json["production_countries"]]),
                      "language": ';'.join([language["english_name"] for language in response_json["spoken_languages"]]),
                      "genre": ';'.join([genre["name"] for genre in response_json["genres"]]),
                      "studios": ';'.join([company["name"] for company in response_json['production_companies']]),
                      "budget": response_json['budget'],
                      "revenue": response_json["revenue"]}, 
                       ignore_index = True)
        
      else:
        df = df.append({"id": film_id}, ignore_index = True)

  return df

Let's run it on the film IDs we just scraped

In [8]:
df = build_film_df(film_ids)
df.head()

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue
0,tt0111161,The Shawshank Redemption,1994-09-23,142,United States of America,English,Drama;Crime,Castle Rock Entertainment,25000000,28341469
1,tt0468569,The Dark Knight,2008-07-14,152,United Kingdom;United States of America,English;Mandarin,Drama;Action;Crime;Thriller,DC Comics;Legendary Pictures;Syncopy;Isobel Gr...,185000000,1004558444
2,tt1375666,Inception,2010-07-15,148,United Kingdom;United States of America,English;Japanese,Action;Science Fiction;Adventure,Legendary Pictures;Syncopy;Warner Bros. Pictures,160000000,825532764
3,tt0137523,Fight Club,1999-10-15,139,Germany;United States of America,English,Drama,Regency Enterprises;Fox 2000 Pictures;Taurus F...,63000000,100853753
4,tt0109830,Forrest Gump,1994-07-06,142,United States of America,English,Comedy;Drama;Romance,Paramount;The Steve Tisch Company,55000000,677387716


Let's have a look at the null values

In [10]:
df[df['title'].isnull()]

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue
5240,tt9573980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9130,tt8075016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9980,tt13423846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scraping additional data

TMDD API has some limits: it doesn't provide data about the people who worked on a film (directors, writer, actors etc.) and its rating data is inaccurate. We'll thus integrate the data we just got by scraping some more information.

### Custom functions

To implement sound software engineering principles we'll scrape the data by building a function for each type of data we need.

#### Film ID

In [11]:
def scrape_film_id(soup):
    
    try:
        film_id = soup.find("meta", {"property": "imdb:pageConst"}).get("content")
    except:
        return np.nan
    else:
        return film_id

#### Directors

In [12]:
def scrape_director(soup):
    
    try:  
        a_tags = soup.find_all(href = re.compile("tt_ov_dr"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    
        directors = list(set([a.text for a in a_tags]))
    
        directors = ';'.join(directors)
        
    except:
        return np.nan
        
    else:
        return directors

#### Writers

In [13]:
def scrape_writer(soup):
    
    try:
        a_tags = soup.find_all(href = re.compile("tt_ov_wr"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
        
        writers = list(set([a.text for a in a_tags]))
        
        writers = ';'.join(writers)
        
    except:
        return np.nan
        
    else:
        return writers

#### IMDB average rating

In [17]:
def scrape_imdb_rating(soup):
    
    try: 
        pattern = r'(?<="ratingValue":)[\d.]+(?=},"contentRating")'
        string = str(soup.find("script", {"type": "application/ld+json"}))
        
        rating = re.findall(pattern = pattern, string = string)[0]
        
    except:
        return np.nan
    
    else:
        return rating
        

#### IMDB rating count

In [18]:
def scrape_rating_count(soup):
    
    try:
        pattern = r'(?<="ratingCount":)[\d.]+'
        string = str(soup.find("script", {"type": "application/ld+json"}))
        rating_count = re.findall(pattern = pattern, string = string)[0]
        
    except:
        return np.nan
    
    else:
        return rating_count

#### Metascore

In [19]:
def scrape_metascore(soup):
    
    try:
        metascore = soup.find("span", class_="score-meta").text 
    
    except:
        return np.nan
    
    else:
        return metascore

#### User review count

In [20]:
def scrape_user_review_count(soup):
    
    try: 
        pattern = r'(?<="total":)\d+(?=,"__typename":"ReviewsConnection"},"criticReviewsTotal":)'
  
        string = str(soup.find("script", {'id': '__NEXT_DATA__'}))
    
        user_review_count = re.findall(pattern = pattern, string = string)[0]
        
    except:
        return np.nan
    
    else:
        return user_review_count

#### Critic review count

In [21]:
def scrape_critic_review_count(soup):
    
    try:   
        spans = soup.find_all("span", class_="three-Elements") 
        
        string = list(filter(lambda x: 'Critic' in str(x), spans))[0].text
        
        critic_review_count = re.findall(r'\d+', string)[0]
    
    except:
        return np.nan
        
    else:
        return critic_review_count

#### Color

In [22]:
def scrape_color(soup):
    
    try:
        pattern = r'(?<="text":")\w+(?=","attributes":\[\],"__typename":"Coloration"})'
        
        string = str(soup.find("script", {"type": "application/json"}))
    
        color = re.findall(pattern = pattern, string = string)[0]
        
    except:
        return np.nan
    
    else:
        return color

#### Aspect ratio

In [24]:
def scrape_aspect_ratio(soup):
    try:
        pattern = r'(?<="aspectRatio":")[\d.\s:]+'
    
        string = str(soup.find("script", {"type": "application/json"}))
    
        aspect_ratio = re.findall(pattern = pattern, string = string)[0] 
        
    except:
        return np.nan
        
    else:
        return aspect_ratio

### Creating the dataframe

If we send 10 000 requests at once we're likely going to run into problems with IMDB, so let's split the film_urls list into 5 equally-sized chunks and let's scrape each chunk at a time by waiting five minutes before moving to the next one.

In [25]:
def build_scraped_df(urls):

  df = pd.DataFrame(columns = ["id", "director", "writer", "imdb_rating", "imdb_rating_count", "metascore", 
                               "user_review_count", "critic_review_count", "color", "aspect_ratio"])

  chunks = np.array_split(urls, 5)

  for chunk in chunks:
    for url in chunk:
      content = requests.get(url).content
      soup = BeautifulSoup(content)

      df = df.append({"id": scrape_film_id(soup),
                      "director": scrape_director(soup),
                      "writer": scrape_writer(soup),
                      "imdb_rating": scrape_imdb_rating(soup),
                      "imdb_rating_count": scrape_rating_count(soup),
                      "metascore": scrape_metascore(soup),
                      "user_review_count": scrape_user_review_count(soup),
                      "critic_review_count": scrape_critic_review_count(soup),
                      "color": scrape_color(soup),
                      "aspect_ratio": scrape_aspect_ratio(soup)}, 
                      ignore_index = True)

    time.sleep(60*5)

  return df


In [26]:
df2 = build_scraped_df(film_urls)

In [29]:
df2.head(20)

,id,director,writer,actors,imdb_rating,imdb_rating_count,metascore,user_review_count,critic_review_count,color,aspect_ratio
0,tt0111161,Frank Darabont,Frank Darabont;Stephen King,Morgan Freeman;Tim Robbins;Bob Gunton,9.3,2505826,80,9743,190,Color,1.85 : 1
1,tt0468569,Christopher Nolan,David S. Goyer;Christopher Nolan;Jonathan Nolan,Christian Bale;Heath Ledger;Aaron Eckhart,9,2455335,84,7761,425,Color,2.39 : 1
2,tt1375666,Christopher Nolan,Christopher Nolan,Elliot Page;Leonardo DiCaprio;Joseph Gordon-Le...,8.8,2203247,74,29,479,Color,1.33 : 1
3,tt0137523,David Fincher,Chuck Palahniuk;Jim Uhls,Edward Norton;Brad Pitt;Meat Loaf,8.8,1971156,66,4127,366,Color,2.39 : 1
4,tt0109830,Robert Zemeckis,Eric Roth;Winston Groom,Gary Sinise;Tom Hanks;Robin Wright,8.8,1934098,82,2806,164,Color,2.39 : 1
5,tt0110912,Quentin Tarantino,Quentin Tarantino;Roger Avary,John Travolta;Uma Thurman;Samuel L. Jackson,8.9,1933232,94,3377,290,Color,2.39 : 1
6,tt0133093,Lilly Wachowski;Lana Wachowski,Lilly Wachowski;Lana Wachowski,Carrie-Anne Moss;Laurence Fishburne;Keanu Reeves,8.7,1789688,73,3,234,Color,2.39 : 1
7,tt0120737,Peter Jackson,Fran Walsh;Philippa Boyens;J.R.R. Tolkien,Elijah Wood;Ian McKellen;Orlando Bloom,8.8,1752103,92,2,202,Color,2.35 : 1
8,tt0167260,Peter Jackson,Fran Walsh;Philippa Boyens;J.R.R. Tolkien,Elijah Wood;Viggo Mortensen;Ian McKellen,8.9,1730819,94,3941,334,Color,2.39 : 1
9,tt0068646,Francis Ford Coppola,Francis Ford Coppola;Mario Puzo,Marlon Brando;James Caan;Al Pacino,9.2,1727046,100,4726,249,NaN,1.85 : 1


Counting the null values for each column

In [ ]:
df2.apply(lambda x: x.isnull().sum())

Is each id unique?

In [ ]:
df2['id'].duplicated().sum()

## Scraping people data

As mentioned previously, TMDB API doesn't provide much data about the people who work on a film, so we'll need to scrape that data ourselves. These pieces of info aren't located in the regular film pages but in the "Full cast and crew" pages. So before scraping the data we first need to collect all these pages.

In [ ]:
crew_pages = []

for url in film_urls:
        
    content = requests.get(url).content
    soup = BeautifulSoup(content)
    
    a_target = soup.find(href = re.compile("tt_cl_sm"), class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
    
    crew_page = "https://www.imdb.com" + a_target.get("href")
    
    crew_pages.append(crew_page)   
    

### Custom functions

#### Full cast

The names of the artists can be found inside the table with the "cast-list" class: they're the alternative text of the images inside the table.

In [ ]:
def scrape_actors(soup):
    
    try:   
        images = soup.find("table", class_="cast_list").find_all("img") 
        
        actors = [img.get("alt") for img in images]  
        
        actors = ';'.join(actors)
    
    except:
        return np.nan
    
    else:
        return actors

#### Cinematographer, editor, composer, producers, production designer, art director, costume designer

The following function scrapes the names of different types of artists. The type must be specified in the 'artist' argument. 

In the "Full Cast & Crew" pages each artist type has its own table element containing the names of the artists. Each of these tables is preceded by an h4 element with an id equal to the artist type. For example, the name of the cinematographer is contained in the table preceded by the h4 element with id equal to "cinematographer". To scrape this data, we'll first access the h4 element, then we'll access the table next to it and finally we'll get all the names.

In [ ]:
def scrape_artist(soup, artist):
    
    try:
        h = soup.find("h4", id = artist)
        
        a_tags = h.find_next("table").find_all("a")
        
        artists = [a.text.lstrip().replace('\n', '') for a in a_tags]
        
        artists = ';'.join(artists)
    
    except:
        return np.nan
        
    else:     
        return artists

### Creating the dataframe

Again we'll split the list of urls into 5 equally-sized chunks and wait 5 mins between each chunk so as not to clutter the IMDB website

In [ ]:
def build_scraped_df_2(urls):
    
    df = pd.DataFrame(columns = ["id", "actors", "cinematographer", "editor", "composer", "producers", "production_designer",
                                "art_director", "costume_designer"])
    
    chunks = np.array_split(urls, 5)

    for chunk in chunks:
        for url in chunk:
            content = requests.get(url).content
            soup = BeautifulSoup(content)
            
            df = df.append({"id": re.findall(r"tt\d+", url)[0],
                       "actors": scrape_actors(soup),
                       "cinematographer": scrape_artist(soup, "cinematographer"),
                       "editor": scrape_artist(soup, "editor"),
                       "composer": scrape_artist(soup, "composer"),
                       "producers": scrape_artist(soup, "producer"),
                       "production_designer": scrape_artist(soup, "production_designer"),
                       "art_director": scrape_artist(soup, "art_director"),
                       "costume_designer": scrape_artist(soup, "costume_designer")},
                       ignore_index = True)
        
        time.sleep(60*5)
        
    return df

Let's run it on our list of 'Full cast and crew' pages

In [ ]:
df3 = build_scraped_df_2(crew_pages)

In [ ]:
df3.head(10)

# Data cleaning

## Joining the dataframes

## Casting the variables

# Load the data onto a database